In [4]:
# Modules to support development
import os
import re
import collections
import itertools
import functools
import logging
import pprint
import numpy as np
import heapq

In [79]:
class InfiniteGrid(collections.defaultdict):
    def __init__(self, default='.'):
        super().__init__(lambda: collections.defaultdict(lambda: default))
        self.dimensions_x = None
        self.dimensions_y = None
        
    def __setitem__(self, pos, val):
        if type(pos) is int:
            return super().__setitem__(pos, val)
        
        yy, xx = pos

        if self.dimensions_x is None:
            self.dimensions_x = (xx, xx)
        else:
            self.dimensions_x = (
                min(xx, self.dimensions_x[0]),
                max(xx, self.dimensions_x[1])
            )
        
        if self.dimensions_y is None:
            self.dimensions_y = (yy, yy)
        else:
            self.dimensions_y = (
                min(yy, self.dimensions_y[0]),
                max(yy, self.dimensions_y[1])
            )    
            

        super().__getitem__(yy)[xx] = val

    def __getitem__(self, pos):
        if type(pos) is int:
            return super().__getitem__(pos)

        yy, xx = pos
        return super().__getitem__(yy)[xx]

    def iterrow(self, row):
        row_data = self[row]
        for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
            yield row_data[xx]

    def enumeraterow(self, row):
        row_data = self[row]
        for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
            yield (row, xx), row_data[xx]

    def __str__(self):
        if self.dimensions_x is None or self.dimensions_y is None:
            return ""

        res = []
        for yy in range(self.dimensions_y[0], self.dimensions_y[1]+1):
            res.append("%02d " % yy)
            for xx in range(self.dimensions_x[0], self.dimensions_x[1]+1):
                 res.append(str(self[yy][xx]))
            res.append("\n")

        return "".join(res)  

def read_input(puzzle_input, part2=False):
    with open(puzzle_input) as ff:
        dd = ff.readlines()

    field = InfiniteGrid('.')
    
    pairs = set()
    
    for ll in dd:
        mm = re.match("Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)", ll)
        if mm:
            sensor_x, sensor_y = [int(xx) for xx in mm.group(1,2)]
            beacon_x, beacon_y = [int(xx) for xx in mm.group(3,4)]

            sensor_x, sensor_y = [int(xx) for xx in mm.group(1,2)]
            field[sensor_y, sensor_x] = 'S'
            field[beacon_y, beacon_x] = 'B'

            pairs.add(((sensor_x, sensor_y), (beacon_x, beacon_y)))

    return field, pairs

def test_read_input():
    field, _ = read_input(os.path.join(os.path.join("..", "dat", "day15_test.txt")))
    assert(field[0, 2]) == 'S'
    assert(field[15, -2]) == 'B'
    print(field)
    
def eliminate(field, sensor, beacon,):
    manhattan_distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
    for yy in range(-manhattan_distance, manhattan_distance+1):
        tmp = manhattan_distance - abs(yy)
        for xx in range(-tmp, tmp+1):
            if field[sensor[1]+yy, sensor[0]+xx] == '.':
                field[sensor[1]+yy, sensor[0]+xx] = '#'

def possible(field, sensor, beacon, search=0):
    possible = set()
    manhattan_distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
    for yy in range(-manhattan_distance, manhattan_distance+1):
        tmp = manhattan_distance - abs(yy) + 1
        #field[sensor[1]+yy, sensor[0]+tmp] = '#'
        #field[sensor[1]+yy, sensor[0]-tmp] = '#'

        possible.add((sensor[0]+tmp, sensor[1]+yy))

    return possible

def test_eliminate():
    field, _ = read_input(os.path.join(os.path.join("..", "dat", "day15_test.txt")))

    eliminate(field, (8,7), (2,10))
    assert(field[-2, 8]) == '#'
    assert(field[-2, 7]) == '.'
    assert(field[-2, 9]) == '.'
    print(field)

def test_possible():
    field, _ = read_input(os.path.join(os.path.join("..", "dat", "day15_test.txt")))

    possible(field, (8,7), (2,10))
    print(field)

#test_read_input()
test_eliminate()
test_possible()


-2 ..........#.................
-1 .........###................
00 ....S...#####...............
01 .......#######........S.....
02 ......#########S............
03 .....###########SB..........
04 ....#############...........
05 ...###############..........
06 ..#################.........
07 .#########S#######S#........
08 ..#################.........
09 ...###############..........
10 ....B############...........
11 ..S..###########............
12 ......#########.............
13 .......#######..............
14 ........#####.S.......S.....
15 B........###................
16 ..........#SB...............
17 ................S..........B
18 ....S.......................
19 ............................
20 ............S......S........
21 ............................
22 .......................B....

00 ....S.......................
01 ......................S.....
02 ...............S............
03 ................SB..........
04 ............................
05 ............................
06 ....

In [73]:
def trival_part1(row, puzzle_input):
    field, pairs = read_input(puzzle_input)
    for sensor, beacon in pairs:
        eliminate(field, sensor, beacon)

    ans = 0
    for v in field.iterrow(row):
        if v == '#':
            ans += 1

    return ans

def part1(row, puzzle_input):
    field, pairs = read_input(puzzle_input)

    ans = 0
    for sensor, beacon in pairs:
        manhattan_distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
        row_distance = abs(sensor[1] - row)
        if row_distance > manhattan_distance:
            continue
        
        tmp = manhattan_distance - abs(row_distance)
        for xx in range(-tmp, tmp+1):
            if field[row, sensor[0]+xx] == '.':
                field[row, sensor[0]+xx] = '#'
                ans += 1

    return ans

def test_part1():
    ans = trival_part1(10, os.path.join(os.path.join("..", "dat", "day15_test.txt")))
    assert ans == 26

    ans = part1(10, os.path.join(os.path.join("..", "dat", "day15_test.txt")))
    assert ans == 26

test_part1()

ans = part1(2000000, os.path.join(os.path.join("..", "dat", "day15.txt")))
print(ans)


5176944


In [110]:
def part2(search, puzzle_input):
    field, pairs = read_input(puzzle_input)

    xx, yy = 0, 0
    seen_points = set()
    possible_points = set()
    for sensor, beacon in pairs:
        manhattan_distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1]) + 1
        for row in range(-manhattan_distance, manhattan_distance+1):
            tmp = manhattan_distance - abs(row)

            point = (sensor[0]+tmp, sensor[1]+row)
            if 0 <= point[0] <= search and 0 <= point[1] <= search:
                if point in seen_points:
                    possible_points.add(point)
                else:
                    seen_points.add(point)

            point = (sensor[0]-tmp, sensor[1]+row)
            if 0 <= point[0] <= search and 0 <= point[1] <= search:
                if point in seen_points:
                    possible_points.add(point)
                else:
                    seen_points.add(point)

    print("Found %s possible points", len(possible_points))
    for point in possible_points:
        found = True
        if 0 <= point[0] <= search and 0 <= point[1] <= search:
            for sensor, beacon in pairs:
                beacon_distance = abs(sensor[0] - beacon[0]) + abs(sensor[1] - beacon[1])
                point_distance = abs(sensor[0] - point[0]) + abs(sensor[1] - point[1])

                if point_distance <= beacon_distance:
                    found = False
        else:
            found = False

        if found:
            print("FOUND", point)
            xx, yy = point
            break

    print("Found %s possible points", len(possible_points))
    return xx*4000000 + yy

def test_part2():
    ans = part2(20, os.path.join(os.path.join("..", "dat", "day15_test.txt")))
    assert ans == 56000011

test_part2()

ans = part2(4000000, os.path.join(os.path.join("..", "dat", "day15.txt")))
print(ans)


Found %s possible points 55
FOUND (14, 11)
Found %s possible points 55
Found %s possible points 2733371
FOUND (3337614, 2933732)
Found %s possible points 2733371
13350458933732
